In [15]:
import pandas as pd
import numpy as np
from requests import get
from pandas import json_normalize
import time

In [12]:
hospitals = pd.read_csv('/Users/patricknaylor/desktop/CA_traffic/Data/health_facility_locations.csv', engine = 'python')
collision_loc = pd.read_csv('/Users/patricknaylor/desktop/CA_traffic/Data/position_df.csv')
kill_res = np.loadtxt('/Users/patricknaylor/desktop/CA_traffic/Data/killed_res.csv', delimiter=',')
kill_res_df = pd.DataFrame(data=kill_res, columns=['killed'])

In [7]:
def get_elevation(lat = None, long = None):
    '''
        script for returning elevation in m from lat, long
    '''
    if lat is None or long is None: return None
    
    query = ('https://api.open-elevation.com/api/v1/lookup'
             f'?locations={lat},{long}')
    
    # Request with a timeout for slow responses
    r = get(query, timeout = 20)

    # Only get the json response in case of 200 or 201
    if r.status_code == 200 or r.status_code == 201:
        elevation = json_normalize(r.json(), 'results')['elevation'].values[0]
    else: 
        elevation = None
    return elevation

In [23]:
time1 = time.perf_counter()
alts = []
for i in range(100):
    alts.append(get_elevation(collision_loc['latitude'][i], collision_loc['longitude'][i]))
    
time2 = time.perf_counter()
print(str(time2 - time1) + "Seconds")
print(alts)

126.58619470798294Seconds
[374, 374, 331, 372, 328, 338, 337, 333, 369, 339, 311, 322, 345, 418, 219, 74, 141, 372, 106, 385, 388, 318, 360, 345, 347, 372, 307, 339, 444, 316, 437, 334, 359, 369, 363, 351, 318, 306, 434, 382, 440, 362, 328, 332, 439, 331, 338, 359, 373, 360, 374, 331, 31, 31, 44, 24, 68, 38, 320, 8, 0, 25, 31, 8, 18, 10, 16, 25, 7, 8, 14, 31, 46, 5, 29, 42, 17, 25, 9, 29, 12, 11, 23, 21, 21, 37, 6, 27, 26, 31, 25, 34, 38, 22, 27, 4, 38, 19, 479, 14]


In [25]:
(((((time2 - time1)/60)/60)/24)/365) * 100000

0.401402190220646

In [ ]:
collision_loc['altitude'] = collision_loc.apply(lambda x: get_elevation(x['latitude'], x['longitude']),axis=1)

In [18]:
print(collision_loc)

         Unnamed: 0  latitude  longitude
0                 0  34.14618 -117.29409
1                 1  34.14618 -117.29409
2                 2  34.11860 -117.27870
3                 3  34.13584 -117.24366
4                 4  34.11574 -117.26098
...             ...       ...        ...
2693990     2693990  38.81863 -121.26125
2693991     2693991  38.79530 -121.26479
2693992     2693992  34.10416 -117.28154
2693993     2693993  34.07961 -117.29416
2693994     2693994  34.10834 -117.28266

[2693995 rows x 3 columns]
